# 第11章时间序列

## 11.1日期和时间数据的类型及工具 

python标准库包含了日期和时间数据的类型，datetime,time,calendar 模块是开始处理时间数据的主要内容。datetime.datetime 类型简写为datetime,是广泛使用的。  

In [1]:
from datetime import datetime

In [2]:
now=datetime.now()

In [3]:
now

datetime.datetime(2019, 7, 17, 0, 41, 51, 631803)

In [4]:
now.year,now.month,now.day

(2019, 7, 17)

####  timedelta 表示两个datetime 对象的时间差：

In [5]:
delta=datetime(2011,1,7)-datetime(2008,6,24,8,15)
delta

datetime.timedelta(days=926, seconds=56700)

In [6]:
delta.days

926

In [7]:
delta.seconds

56700

你可以为一个datetime对象加上（或减去）一个timedelta 或其整数倍来产生一个新的datetime 对象：

In [8]:
from datetime import timedelta

In [9]:
start=datetime(2011,1,7)

In [11]:
start+timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [12]:
start-2*timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

### 11.1.1字符串与datetime 互相转换 

 你可以使用str 方法或传递一个指定的格式给strftime 方法来对datetime对象和pandas 的Timestamp对象进行格式化

In [13]:
stamp=datetime(2011,1,3)

In [14]:
str(stamp)

'2011-01-03 00:00:00'

In [15]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

你可以使用datetime.srtptime和这些格式代码 ，将字符串转换为日期：

In [16]:
value='2011-01-03'

In [17]:
datetime.strptime(value,'%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [18]:
datestrs=['7/6/2011','8/6/2011']

In [20]:
[datetime.strptime(x,'%m/%d/%Y')for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

datetime.strptime 是在已知格式的情况下转换日期的好方法。然而，每次都必须编写一个格式代码可能有点犯人，特别是对通用日期格式。在这种情况下，你可以使用 第三方dateutil 包的parser.parse 方法

In [21]:
from dateutil.parser import parse

In [22]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

dateutil能够解析大部分人类可理解的日期表示：

In [23]:
parse('Jan 31,1997 10:45 PM')

datetime.datetime(2019, 1, 31, 22, 45)

In [25]:
# 在国际场合，日期出现在月份之前时，传递 dayfirst=True 来表明这种情况：
parse('6/11/2011',dayfirst=True)

datetime.datetime(2011, 11, 6, 0, 0)

#### pandas 主要是面向处理日期数组的，无论是用作轴索引还是用作DateFrame 中的列。
#### to_datetime 方法可以转换很多不同的日期表示格式。标准日期格式，可以非常快的转换：

In [28]:
import pandas as pd
import numpy as np

In [26]:
datestrs=['2011-07-06 12:00:00','2011-08-06 00:00:00']

In [29]:
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

#### to_datetime 方法还可以处理那些被认为是缺失值的值（None,空字符串等） 

In [30]:
idx=pd.to_datetime(datestrs+[None])

In [31]:
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [32]:
idx[2]

NaT

In [34]:
# NaT(Not a time)是pandas 中时间戳数据的null 值
pd.isnull(idx)

array([False, False,  True])

## 11.2 时间序列基础 

#### pandas 中的基础时间序列种类是由时间戳索引的Series,在pandas  外部则通常表示为 Python 字符串或datetime 对象

In [35]:
from datetime import datetime

In [36]:
dates=[datetime(2011,1,2),datetime(2011,1,5),
       datetime(2011,1,7),datetime(2011,1,8),
      datetime(2011,1,10),datetime(2011,1,12)]

In [37]:
ts=pd.Series(np.random.randn(6),index=dates)

In [38]:
ts

2011-01-02    0.225582
2011-01-05    0.030175
2011-01-07    0.622708
2011-01-08   -1.731421
2011-01-10    0.977405
2011-01-12    0.196032
dtype: float64

In [39]:
# 在这种情况下，这些datetime 对象可以被放入DatetimeIndex 中
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [41]:
# 在其他Series 类似，不同索引的时间序列之间的算术运算在日期上自动对齐：
ts+ts[::2]

2011-01-02    0.451164
2011-01-05         NaN
2011-01-07    1.245416
2011-01-08         NaN
2011-01-10    1.954809
2011-01-12         NaN
dtype: float64

In [43]:
ts[::2]   #将ts中每隔一个的元素选择出

2011-01-02    0.225582
2011-01-07    0.622708
2011-01-10    0.977405
dtype: float64

In [45]:
#pandas 使用Numpy 的datetime64 数据类型在纳秒级分辨率下存储时间戳：
ts.index.dtype

dtype('<M8[ns]')

In [46]:
# DatetimeIndex 中的标量值是pandas 的Timestamp 对象
stamp=ts.index[0]

In [47]:
stamp

Timestamp('2011-01-02 00:00:00')

In [48]:
type(stamp)

pandas._libs.tslibs.timestamps.Timestamp

###  11.2.1索引，选择，子集 

####  当你基于标签对行索引和选择时，时间序列的行为和其他的pandas.Series类似：

In [49]:
ts

2011-01-02    0.225582
2011-01-05    0.030175
2011-01-07    0.622708
2011-01-08   -1.731421
2011-01-10    0.977405
2011-01-12    0.196032
dtype: float64

In [51]:
stamp=ts.index[2]
stamp

Timestamp('2011-01-07 00:00:00')

In [52]:
ts[stamp]

0.6227080791957202

In [54]:
# 为了方便，你还可以传递一个能解释为日期的字符串：
ts['1/10/2011']

0.977404646156787

In [55]:
ts['20110110']

0.977404646156787

#### 对于一个长的时间序列，可以传递一个年份或一个年份和月份来轻松地选择数据地切片： 

In [57]:
longer_ts=pd.Series(np.random.randn(1000),index=pd.date_range('1/1/2000',periods=1000))
longer_ts

2000-01-01   -0.471389
2000-01-02   -0.227970
2000-01-03    0.035325
2000-01-04   -1.002364
2000-01-05   -0.111684
2000-01-06   -1.338192
2000-01-07    0.945218
2000-01-08   -1.242340
2000-01-09   -0.493336
2000-01-10   -0.348309
2000-01-11   -0.301978
2000-01-12   -0.031278
2000-01-13   -1.198918
2000-01-14    0.930574
2000-01-15    0.157608
2000-01-16    0.792701
2000-01-17    2.339237
2000-01-18    0.201820
2000-01-19   -0.348850
2000-01-20   -0.446086
2000-01-21    0.064341
2000-01-22   -1.097421
2000-01-23   -1.482211
2000-01-24    1.707531
2000-01-25    0.377977
2000-01-26    0.436404
2000-01-27   -0.288229
2000-01-28    0.679765
2000-01-29    0.774107
2000-01-30   -0.437865
                ...   
2002-08-28    1.633318
2002-08-29    0.580055
2002-08-30    1.136198
2002-08-31    0.174042
2002-09-01    0.667690
2002-09-02   -0.749656
2002-09-03    0.297459
2002-09-04    0.430921
2002-09-05    0.582600
2002-09-06   -0.285745
2002-09-07   -0.333984
2002-09-08   -1.405377
2002-09-09 

In [58]:
longer_ts['2001']

2001-01-01   -0.129443
2001-01-02   -0.190576
2001-01-03   -1.357558
2001-01-04    0.516718
2001-01-05   -1.366844
2001-01-06   -0.509174
2001-01-07    1.528841
2001-01-08    0.947286
2001-01-09   -0.424125
2001-01-10    1.549906
2001-01-11   -0.358217
2001-01-12   -0.887594
2001-01-13    0.509967
2001-01-14    0.221425
2001-01-15    0.150494
2001-01-16   -0.912626
2001-01-17   -0.027452
2001-01-18   -0.296567
2001-01-19   -0.786649
2001-01-20   -0.810580
2001-01-21   -0.089048
2001-01-22    0.687309
2001-01-23    0.436866
2001-01-24   -0.604104
2001-01-25    1.261637
2001-01-26   -1.676009
2001-01-27   -0.640823
2001-01-28   -0.507124
2001-01-29   -0.276101
2001-01-30   -0.841603
                ...   
2001-12-02   -0.334514
2001-12-03    0.707828
2001-12-04   -1.856967
2001-12-05   -0.819696
2001-12-06   -1.363994
2001-12-07   -1.280821
2001-12-08    1.106494
2001-12-09    1.977086
2001-12-10    0.382184
2001-12-11   -1.506062
2001-12-12    2.192540
2001-12-13   -0.213464
2001-12-14 

In [59]:
# 指定一个月份也是有效的：
longer_ts['2001-05']

2001-05-01    0.092282
2001-05-02   -0.181187
2001-05-03   -1.532532
2001-05-04   -0.510696
2001-05-05    0.501986
2001-05-06   -0.870267
2001-05-07   -0.887428
2001-05-08   -0.323494
2001-05-09   -0.499175
2001-05-10   -1.245047
2001-05-11   -0.306729
2001-05-12   -0.677119
2001-05-13    0.063046
2001-05-14   -0.435970
2001-05-15    1.116718
2001-05-16    0.158761
2001-05-17    1.981044
2001-05-18   -0.451095
2001-05-19    1.759092
2001-05-20    0.620857
2001-05-21   -1.488943
2001-05-22    1.342986
2001-05-23   -1.547162
2001-05-24    0.084213
2001-05-25   -0.461266
2001-05-26   -0.062168
2001-05-27    0.889242
2001-05-28    0.020141
2001-05-29   -0.697574
2001-05-30    0.661402
2001-05-31    1.916235
Freq: D, dtype: float64

In [60]:
# 使用datetime 对象进行切片也是可以的
ts[datetime(2011,1,7):]

2011-01-07    0.622708
2011-01-08   -1.731421
2011-01-10    0.977405
2011-01-12    0.196032
dtype: float64

In [61]:
# 因为大部分的时间序列数据是按照 时间序列排序的，你可以使用不包含在时间序列中的时间戳进行切片，以执行范围查询：

In [62]:
ts

2011-01-02    0.225582
2011-01-05    0.030175
2011-01-07    0.622708
2011-01-08   -1.731421
2011-01-10    0.977405
2011-01-12    0.196032
dtype: float64

In [63]:
ts['1/6/2011':'1/11/2011']

2011-01-07    0.622708
2011-01-08   -1.731421
2011-01-10    0.977405
dtype: float64

In [65]:
ts.truncate(after='1/9/2011')  # 使用truncate 也可以

2011-01-02    0.225582
2011-01-05    0.030175
2011-01-07    0.622708
2011-01-08   -1.731421
dtype: float64

In [66]:
ts

2011-01-02    0.225582
2011-01-05    0.030175
2011-01-07    0.622708
2011-01-08   -1.731421
2011-01-10    0.977405
2011-01-12    0.196032
dtype: float64

In [69]:
# 上面这些操作也都适用于DataFrame ,并在其行上进行索引：
dates=pd.date_range('1/1/2000',periods=100,freq='W-WED')

In [70]:
long_df=pd.DataFrame(np.random.randn(100,4),index=dates,columns=['Colorado','Texas','New York','Ohip'])

In [72]:
long_df[0:5]

,Colorado,Texas,New York,Ohip
2000-01-05,0.215490,-0.550301,-1.852813,-0.567559
2000-01-12,-0.690927,-1.347171,-0.423750,-0.280429
2000-01-19,-0.516773,-0.841020,0.246633,1.718928
2000-01-26,-0.898328,1.204434,0.183939,-0.290027
2000-02-02,1.676237,1.078234,-1.299463,0.530784


In [73]:
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohip
2001-05-02,-0.958397,2.577794,1.871249,0.621948
2001-05-09,-1.084049,-0.131846,1.760172,-0.087050
2001-05-16,-0.467709,-0.820876,0.535854,-2.511390
2001-05-23,0.109317,-0.187961,-0.301959,0.193084
2001-05-30,-0.254868,0.532119,-0.375941,1.606671


### 11.2.2 含有重复索引的时间序列

In [75]:
# 在某些应用中，可能会有多个数据观察值落在特定的时间戳上。下面是一个例子：
dates=pd.DatetimeIndex(['1/1/2000','1/2/2000','1/2/2000','1/2/2000','1/3/2000'])
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-02', '2000-01-02',
               '2000-01-03'],
              dtype='datetime64[ns]', freq=None)

In [77]:
dup_ts=pd.Series(np.arange(5),index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

In [78]:
# 通过检查索引的is_unique属性，我们可以看出索引并不是唯一的：
dup_ts.index.is_unique

False

In [80]:
# 对上面的Series 进行索引，结果是标量值还是Series切片取决于是否有时间戳是重复的：
dup_ts['1/3/2000']

4

In [81]:
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

#### 假设你想要聚合含有非唯一时间戳数据，一种方式就是使用groupby 并传递level=0: 

In [82]:
grouped=dup_ts.groupby(level=0)

In [83]:
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

In [85]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3日期范围、频率和移位

#### pandas拥有一整套标准的时间序列频率和工具用于重新采样，推断频率以及生成固定频率的数据范围。例如，你可以通过调用resample
#### 方法将样本时间序列转换为固定的每日频率数据：

In [86]:
ts

2011-01-02    0.225582
2011-01-05    0.030175
2011-01-07    0.622708
2011-01-08   -1.731421
2011-01-10    0.977405
2011-01-12    0.196032
dtype: float64

In [89]:
resampler=ts.resample('D')  # 'D'被解释为每日频率
resampler

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

### 11.3.1 生成日期范围

In [92]:
# pandas.date_range 是用于根据特定频率生成指定长度的DatetimeIndex:
index=pd.date_range('2012-04-01','2012-06-01')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [93]:
# 默认情况下，date_range 生成的是每日时间戳。如果你只传递一个起始或结尾日期，你必须传递一个用于生成范围的数字
pd.date_range(start='2012-04-01',periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [94]:
pd.date_range(end='2012-06-01',periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

In [96]:
#例如你需要一个包含每月最后业务日期的时间索引，你可以传递‘BM’频率（business end of month）
pd.date_range('2000-01-01','2000-12-01',freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

In [97]:
# 默认情况下，date_range保留开始或结束时间戳的时间（如果有的话）
pd.date_range('2012-05-02 12:56:31',periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

In [98]:
# 有时候你会获得包含时间信息的开始日期或结束日期，但是你想要生成的是标准化为零点的时间戳。有一个normalize 选项可以实现这个功能：
pd.date_range('2012-05-02 12:56:31',periods=5,normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 11.3.2 频率和日期偏置

pandas 中的频率是由基础频率和倍数构成的。基础频率通常会有字符串别名，例如“M”代表每月，“H”代表每小时。对于每个基础频率，都有一个对象可以被用于定义日期偏置。例如，每小时的频率可以使用Hour 类来表示：

In [99]:
from pandas.tseries.offsets import Hour ,Minute
hour=Hour()

In [100]:
hour

<Hour>

In [102]:
# 你可以传递一个整数来定义偏置量的倍数：
four_hours=Hour(4)
four_hours

<4 * Hours>

####  在大多数应用中，你都不需要显式地创建这些对象，而是使用字符串别名，如‘H’或‘4H’。在基础频率前放一个整数就可以生成倍数：

In [103]:
pd.date_range('2000-01-01','2000-01-03 23:59',freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

In [104]:
#多个偏置可以通过加法进行联合：
Hour(2)+Minute(30)

<150 * Minutes>

In [105]:
#类似地，你可以传递频率字符串，例如‘1h30min’将会有效地转换为同等地表达式：
pd.date_range('2000-01-01',periods=10,freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

####  有些频率描述点地时间并不是均匀分隔。我们将这些日期称为锚定偏置量，如‘M’日历月末，‘BM’月内最后工作日

#### 11.3.2.1 月中某星期的日期 ，以WOM开始，它允许你可以获得每月第三个星期五这样的日期：

In [106]:
rng=pd.date_range('2012-01-01','2012-09-01',freq='WOM-3FRI')

In [107]:
rng

DatetimeIndex(['2012-01-20', '2012-02-17', '2012-03-16', '2012-04-20',
               '2012-05-18', '2012-06-15', '2012-07-20', '2012-08-17'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

In [108]:
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

### 11.3.3移位（前向和后向）日期

#### “移位”是指将日期按照时间向前移动或向后移动。Series 和DataFrame 都有一个shift 方法用于进行简单的前向或后向移位，而不改变索引

In [109]:
ts=pd.Series(np.random.randn(4),index=pd.date_range('1/1/2000',periods=4,freq='M'))

In [110]:
ts

2000-01-31   -0.086784
2000-02-29   -2.347511
2000-03-31   -1.633678
2000-04-30    0.892110
Freq: M, dtype: float64

In [111]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.086784
2000-04-30   -2.347511
Freq: M, dtype: float64

In [112]:
ts.shift(-2)

2000-01-31   -1.633678
2000-02-29    0.892110
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

In [ ]:
# 当我们像上面这样进行移位时，会在时间序列的起始位置或结束位引入缺失值。
# shift 常用于计算时间序列或dataframe 多列时间序列的百分比变化，代码实现如下：
ts/ts.shift(1)-1

In [113]:
# 由于简单移位并不改变索引，一些数据会被丢失。因此，如果频率已知，则可以将频率传递给shift 来推移时间戳而不是简单的数据：
ts.shift(2,freq='M')

2000-03-31   -0.086784
2000-04-30   -2.347511
2000-05-31   -1.633678
2000-06-30    0.892110
Freq: M, dtype: float64

In [114]:
# 其他的频率也可以传递，为你前移和后移数据提供灵活性：
ts.shift(3,freq='D')

2000-02-03   -0.086784
2000-03-03   -2.347511
2000-04-03   -1.633678
2000-05-03    0.892110
dtype: float64

In [115]:
ts.shift(1,freq='90T')   #T代表分钟

2000-01-31 01:30:00   -0.086784
2000-02-29 01:30:00   -2.347511
2000-03-31 01:30:00   -1.633678
2000-04-30 01:30:00    0.892110
Freq: M, dtype: float64

####  11.3.3.1 使用偏置进行移位日期 

pandas 日期偏置也可以使用datetime 或Timestamp 完成

In [116]:
from pandas.tseries.offsets import Day,MonthEnd

In [117]:
now=datetime(2011,11,17)

In [118]:
now+3*Day()

Timestamp('2011-11-20 00:00:00')

In [119]:
# 如果你添加了一个锚定偏置量，比如MonthEnd,根据频率规则，第一个增量会将日期“前滚”到下一个日期：
now+MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [120]:
now+MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

锚定偏置可以使用rollforward 和rollback 分别显式地将日期向前或向后“滚动” ；

In [121]:
offset=MonthEnd()

In [122]:
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [123]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

将移位方法与groupby 一起使用是日期偏置的一种创造性用法：

In [124]:
ts=pd.Series(np.random.randn(20),index=pd.date_range('1/15/2000',periods=20,freq='4d'))

In [125]:
ts

2000-01-15    0.794958
2000-01-19    0.009039
2000-01-23   -3.072812
2000-01-27   -0.391254
2000-01-31   -0.265432
2000-02-04   -0.588617
2000-02-08    1.004156
2000-02-12   -1.889233
2000-02-16    0.359643
2000-02-20    0.178083
2000-02-24    0.743377
2000-02-28    0.718740
2000-03-03   -0.070548
2000-03-07   -2.114996
2000-03-11   -0.369946
2000-03-15   -0.517419
2000-03-19   -1.323584
2000-03-23   -0.847188
2000-03-27    1.877267
2000-03-31    0.546246
Freq: 4D, dtype: float64

In [126]:
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.585100
2000-02-29    0.075164
2000-03-31   -0.352521
dtype: float64

In [127]:
# 当然使用resample 是更简单更快捷的方法
ts.resample('M').mean()

2000-01-31   -0.585100
2000-02-29    0.075164
2000-03-31   -0.352521
Freq: M, dtype: float64

## 11.4 时区处理 

In [128]:
import pytz 

In [129]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [130]:
#要获得pytz 的时区对象，可使用pytz.timezone:
tz=pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

### 11.4.1时区的本地化和转换

In [131]:
rng=pd.date_range('3/9/2012 9:30',periods=6,freq='D')

In [132]:
rng

DatetimeIndex(['2012-03-09 09:30:00', '2012-03-10 09:30:00',
               '2012-03-11 09:30:00', '2012-03-12 09:30:00',
               '2012-03-13 09:30:00', '2012-03-14 09:30:00'],
              dtype='datetime64[ns]', freq='D')

In [133]:
ts=pd.Series(np.random.randn(len(rng)),index=rng)

In [134]:
ts

2012-03-09 09:30:00   -1.056533
2012-03-10 09:30:00   -0.815299
2012-03-11 09:30:00   -0.664621
2012-03-12 09:30:00   -0.276742
2012-03-13 09:30:00    2.993820
2012-03-14 09:30:00    1.739494
Freq: D, dtype: float64

In [135]:
# 索引的tz 属性是None:
print(ts.index.tz)

None


In [136]:
# 日期范围可以通过时区集合来生成：
pd.date_range('3/9/2012 9:30',periods=10,freq='D',tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [137]:
# 使用tz_localize 方法可以将简单时区转换到本地化时区：
ts_utc=ts.tz_localize('UTC')

In [138]:
ts_utc

2012-03-09 09:30:00+00:00   -1.056533
2012-03-10 09:30:00+00:00   -0.815299
2012-03-11 09:30:00+00:00   -0.664621
2012-03-12 09:30:00+00:00   -0.276742
2012-03-13 09:30:00+00:00    2.993820
2012-03-14 09:30:00+00:00    1.739494
Freq: D, dtype: float64

In [140]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

In [141]:
# 一旦时间序列被本地化为某个特定的时区，则可以通过tz_convert 将其转换为另一个时区；

In [142]:
ts_utc.tz_convert('America/New_York')

2012-03-09 04:30:00-05:00   -1.056533
2012-03-10 04:30:00-05:00   -0.815299
2012-03-11 05:30:00-04:00   -0.664621
2012-03-12 05:30:00-04:00   -0.276742
2012-03-13 05:30:00-04:00    2.993820
2012-03-14 05:30:00-04:00    1.739494
Freq: D, dtype: float64

In [144]:
# 在之前跨越America/New-York 时区的DST 转换时间序列的例子中，我们可以将其本地化到EST 并转换为UTC或柏林时间：
ts_eastern=ts.tz_localize('America/New_York')
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00   -1.056533
2012-03-10 14:30:00+00:00   -0.815299
2012-03-11 13:30:00+00:00   -0.664621
2012-03-12 13:30:00+00:00   -0.276742
2012-03-13 13:30:00+00:00    2.993820
2012-03-14 13:30:00+00:00    1.739494
Freq: D, dtype: float64

In [145]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00   -1.056533
2012-03-10 15:30:00+01:00   -0.815299
2012-03-11 14:30:00+01:00   -0.664621
2012-03-12 14:30:00+01:00   -0.276742
2012-03-13 14:30:00+01:00    2.993820
2012-03-14 14:30:00+01:00    1.739494
Freq: D, dtype: float64

In [147]:
# tz_localize 和tz_convert 也是DatetimeIndex的实例方法：
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

### 11.4.2 时区感知时间戳对象的操作

与时间序列和日期范围类似，单独的Timestamp对象也可以从简单时间戳本地化为时区感知时间戳，并从一个时区转换到另一个时区：

In [148]:
stamp=pd.Timestamp('2011-03-12 04:00')

In [149]:
stamp_utc=stamp.tz_localize('utc')

In [150]:
stamp_utc.tz_convert('America/New_York')

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

你也可以在创建Timestamp的时候传递一个时区：

In [151]:
stamp_moscow=pd.Timestamp('2011-03-12 04:00',tz='Europe/Moscow')

In [152]:
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

时区感知的timestamp对象内部储存了一个Unix 纪元（1970年1月1日）至今的纳秒数量UTC 时间戳数值，该数值在时区转换中是不变的。

In [153]:
stamp_utc.value

1299902400000000000

In [154]:
stamp_utc.tz_convert('America/New_York').value

1299902400000000000

在使用pandas 的DateOffset 进行时间算术时，pandas尽可能遵从夏令时。这里我们构建恰好在DST转换之前发生的时间戳（向前和向后）。首先，我们构造转换到DST 之前的30分钟的时间：

In [155]:
from pandas.tseries.offsets import Hour
stamp=pd.Timestamp('2012-03-12 01:30',tz='US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [156]:
stamp+Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

之后我们构建从DST 进行转换前的90分钟：

In [157]:
stamp=pd.Timestamp('2012-11-04 00:30',tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [158]:
stamp+2*Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

###  11.4.3 不同时区间的操作

如果两个时区不同的时间序列需要联合，那么结果将是UTC 时间的。由于时间戳以UTC格式储存，这是一个简单的操作，不需要转换:

In [160]:
rng=pd.date_range('3/7/2012 9:30',periods=10,freq='B')

In [161]:
rng

DatetimeIndex(['2012-03-07 09:30:00', '2012-03-08 09:30:00',
               '2012-03-09 09:30:00', '2012-03-12 09:30:00',
               '2012-03-13 09:30:00', '2012-03-14 09:30:00',
               '2012-03-15 09:30:00', '2012-03-16 09:30:00',
               '2012-03-19 09:30:00', '2012-03-20 09:30:00'],
              dtype='datetime64[ns]', freq='B')

In [162]:
ts=pd.Series(np.random.randn(len(rng)),index=rng)

In [163]:
ts

2012-03-07 09:30:00    2.033830
2012-03-08 09:30:00   -0.465499
2012-03-09 09:30:00   -0.335249
2012-03-12 09:30:00   -0.332141
2012-03-13 09:30:00   -0.081762
2012-03-14 09:30:00    0.071499
2012-03-15 09:30:00    0.060351
2012-03-16 09:30:00   -0.236641
2012-03-19 09:30:00    1.557714
2012-03-20 09:30:00   -0.529929
Freq: B, dtype: float64

In [164]:
ts1=ts[:7].tz_localize('Europe/London')

In [165]:
ts1

2012-03-07 09:30:00+00:00    2.033830
2012-03-08 09:30:00+00:00   -0.465499
2012-03-09 09:30:00+00:00   -0.335249
2012-03-12 09:30:00+00:00   -0.332141
2012-03-13 09:30:00+00:00   -0.081762
2012-03-14 09:30:00+00:00    0.071499
2012-03-15 09:30:00+00:00    0.060351
Freq: B, dtype: float64

In [166]:
ts2=ts1[2:].tz_convertvert('Europe/Moscow')

In [167]:
ts2

2012-03-09 13:30:00+04:00   -0.335249
2012-03-12 13:30:00+04:00   -0.332141
2012-03-13 13:30:00+04:00   -0.081762
2012-03-14 13:30:00+04:00    0.071499
2012-03-15 13:30:00+04:00    0.060351
Freq: B, dtype: float64

In [168]:
result=ts1+ts2

In [169]:
result

2012-03-07 09:30:00+00:00         NaN
2012-03-08 09:30:00+00:00         NaN
2012-03-09 09:30:00+00:00   -0.670499
2012-03-12 09:30:00+00:00   -0.664283
2012-03-13 09:30:00+00:00   -0.163523
2012-03-14 09:30:00+00:00    0.142999
2012-03-15 09:30:00+00:00    0.120702
Freq: B, dtype: float64

In [170]:
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

## 11.5 时间区间和区间算术 

时间区间表示的是时间范围，比如一些天，一个月，一些季度或者是一些年。Period类表示的正是这种数据类型，需要一个字符串或数字以及表11-4 的频率；

In [171]:
p=pd.Period(2007,freq='A-DEC')

In [172]:
p

Period('2007', 'A-DEC')

In [173]:
# 在这个例子里，Period对象表示的是从2007年1月1日到2007年12月31日的时间段，在时间段上增加或减去整数可以方便地根据它们的频率进行移位。
p+5

Period('2012', 'A-DEC')

In [174]:
p-2

Period('2005', 'A-DEC')

In [176]:
# 如果两个区间拥有相同的频率，则它们的差就是它们之间的单位数：
pd.Period('2014',freq='A-DEC')-p


7

In [177]:
# 使用period_range 函数可以构造规则区间序列：
rng=pd.period_range('2000-01-01','2000-06-30',freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

In [178]:
# PeriodsIndex类储存的是区间的序列，可以作为任意pandas 数据结构的轴索引
pd.Series(np.random.randn(6),index=rng)

2000-01    1.481017
2000-02    0.452624
2000-03    1.685122
2000-04   -0.816342
2000-05    1.697601
2000-06   -0.675811
Freq: M, dtype: float64

In [182]:
# 如果你有一个字符串数组，你也可以使用PeriodIndex 类：
values=['2001Q3','2002Q2','2003Q1']
index=pd.PeriodIndex(values,freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

### 11.5.1 区间频率转换 

使用asfreq 可以将区间和PeriodIndex对象转换为其他的频率。例如，假设我们有一个年度区间，并且想要在一年的开始或结束时将其转换位月度区间。

In [183]:
p=pd.Period('2007',freq='A-DEC')

In [184]:
p

Period('2007', 'A-DEC')

In [185]:
p.asfreq('M',how='end')

Period('2007-12', 'M')

In [186]:
p.asfreq('M',how='start')

Period('2007-01', 'M')

当你从高频率向低频率转换时，pandas 根据子区间“所属”来决定父区间，例如，在A-JUN 频率中，Aug-2007是2008区间的一部分：

In [187]:
p=pd.Period('Aug-2007','M')

In [188]:
p.asfreq('A-JUN')

Period('2008', 'A-JUN')

完整的PeriodIndex 对象或时间序列可以按照相同的语义进行转换：

In [189]:
rng=pd.period_range('2006','2009',freq='A-DEC')

In [190]:
ts=pd.Series(np.random.randn(len(rng)),index=rng)

In [191]:
ts

2006    0.539180
2007   -0.443216
2008    0.995524
2009   -0.982499
Freq: A-DEC, dtype: float64

In [192]:
ts.asfreq('M',how='start')

2006-01    0.539180
2007-01   -0.443216
2008-01    0.995524
2009-01   -0.982499
Freq: M, dtype: float64

In [193]:
#  这里，年度区间将被替换成对应于每个年度区间的第一个月的月度区间。
# 如果，我们想要每年最后一个工作日，我们可以使用‘B’频率 来表示我们想要的区间是末端
ts.asfreq('B',how='end')

2006-12-29    0.539180
2007-12-31   -0.443216
2008-12-31    0.995524
2009-12-31   -0.982499
Freq: B, dtype: float64

###  11.5.2季度区间频率

In [194]:
p=pd.Period('2012Q4',freq='Q-JAN')
p

Period('2012Q4', 'Q-JAN')

In [195]:
# 在财年结束于1月的情况下，2012Q4运行时间为11月至1月，你可以通过转换为每日频率来检查
p.asfreq('D','start')

Period('2011-11-01', 'D')

In [196]:
p.asfreq('D','end')

Period('2012-01-31', 'D')

In [197]:
# 因此，做简单的区间算术是可行的，例如，要获取在季度倒数第二个工作日下午4点的时间戳，你可以：
p4pm=(p.asfreq('B','e')-1).asfreq('T','s')+16*60

In [199]:
p4pm

Period('2012-01-30 16:00', 'T')

In [200]:
p4pm.to_timestamp()

Timestamp('2012-01-30 16:00:00')

你可以使用period_range生成季度序列。它的算术也是一样的：

In [201]:
rng=pd.period_range('2011Q3','2012Q4',freq='Q-JAN')

In [204]:
ts=pd.Series(np.arange(len(rng)),index=rng)
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int32

In [205]:
new_rng=(rng.asfreq('B','e')-1).asfreq('T','s')+16*60

In [206]:
ts.index=new_rng.to_timestamp()

In [207]:
ts

2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int32

###  将时间戳转换为区间（以及逆转换）

通过时间戳索引的Series 和DataFrame可以被to_period 方法转换为区间：

In [208]:
rng=pd.date_range('2000-01-01',periods=3,freq='M')

In [ ]:
ts=pd